<a href="https://colab.research.google.com/github/zyazam/FULL-ML-Project/blob/main/All_Project_to_task_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
psparks_instacart_market_basket_analysis_path = kagglehub.dataset_download('psparks/instacart-market-basket-analysis')

print('Data source import complete.')


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
psparks_instacart_market_basket_analysis_path = kagglehub.dataset_download('psparks/instacart-market-basket-analysis')

print('Data source import complete.')


# ***Welcome To The Machine Learning (1) Project***

***What we are trying to do in this project is to apply everything we learned in the machine learning course, from analyzing, cleaning, and processing data to applying machine learning algorithms.***

## And of course, we will use this dataset [**Dataset**](https://www.kaggle.com/datasets/psparks/instacart-market-basket-analysis) to practice sorcery on it.


## ***We will do our best to carry out all the required tasks..***

## **The students in charge of this project:**
- ***Hamza Moeen Rasheed Issa***
- ***Zaid Yusef GH. Hashash***



---






---




- In the first cell, we will import the libraries used in the project.

In [ ]:
import pandas as pd
import numpy as np
import os
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import statsmodels.stats.api as sms
import statsmodels.api as sm
import scipy.stats as stats
from category_encoders import TargetEncoder
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import gc
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, recall_score, precision_score

- Download data directly from Kaggle, via **API Token**

  -**(google.colab files)**
  This is the library used to upload API tokens

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
Types = {
    'aisle_id': 'int32',
    'aisle': 'category',
    'order_id': 'int32',
    'user_id': 'int32',
    'eval_set': 'category',
    'order_number': 'int32',
    'order_dow': 'int32',
    'order_hour_of_day': 'int32',
    'days_since_prior_order': 'float16',
    'department_id': 'int8',
    'department': 'category',
    'product_id': 'int32',
    'product_name': 'category',
    'add_to_cart_order': 'int32',
    'reordered': 'int8',
}


# Reading files using enhanced data types
aisles = pd.read_csv('/kaggle/input/aisles.csv', dtype={i: Types.get(i, None) for i in ['aisle_id','aisle']})

departments = pd.read_csv('/kaggle/input/departments.csv', dtype={i: Types.get(i, None) for i in ['department_id','department']})

products = pd.read_csv('/kaggle/input/products.csv', dtype={i: Types.get(i, None) for i in ['product_id', 'aisle_id', 'department_id',' product_name']})

# Read orders, excluding 'days_since_prior_order' from explicit dtype setting during read_csv
orders = pd.read_csv('/kaggle/input/orders.csv', dtype={i: Types.get(i, None) for i in ['order_id', 'user_id', 'order_number', 'order_dow', 'order_hour_of_day','eval_set']})
# Convert 'days_since_prior_order' to float16 after loading the DataFrame
orders['days_since_prior_order'] = orders['days_since_prior_order'].astype('float16')

order_products_prior = pd.read_csv('/kaggle/input/order_products__prior.csv', dtype={i: Types.get(i, None) for i in ['order_id', 'product_id', 'add_to_cart_order', 'reordered']})


In [ ]:
print(aisles.info())
print("------------------------------------------------")
print(orders.info())
print("------------------------------------------------")
print(departments.info())
print("------------------------------------------------")
print(products.info())
print("------------------------------------------------")
print(order_products_prior.info())

In [ ]:
My_Data = pd.merge(orders, order_products_prior,how = 'left', on='order_id')


My_Data = pd.merge(My_Data, products,how = 'left', on='product_id')


My_Data = pd.merge(My_Data, aisles,how = 'left', on='aisle_id')


My_Data = pd.merge(My_Data, departments,how = 'left', on='department_id')


My_Data.head()

In [ ]:
My_Data['product_name'] = My_Data['product_name'].astype('category')

In [ ]:
print(My_Data.shape)
print("-----------------------------")
print(My_Data.info())

# **• Missing value analysis and visualization**

In [ ]:
# missing_data = My_Data.isnull().sum()
# print(missing_data)

# print("-------------------------------")
# missing_percentage = (My_Data.isnull().sum() / len(My_Data)) * 100
# print(missing_percentage)

In [ ]:
# missing_percentage = missing_percentage[missing_percentage > 0]

# if not missing_percentage.empty:
#     missing_percentage.sort_values().plot(kind='barh', figsize=(10, 6), color='skyblue')
#     plt.title('Percentage of Missing Values in Dataset')
#     plt.xlabel('Percentage (%)')
#     plt.show()
#     plt.close()

# else:
#   print("There are not missing values")

## **We will process missing values ​​directly to reduce memory usage.**

-  ***Since these are missing values ​​due to the lack of matching records in other tables, we will use a fixed-value compensation method.***

In [ ]:
# # Text column processing
# for col in ['product_name', 'aisle', 'department']:
#     if My_Data[col].dtype.name == 'category' and 'Unknown' not in My_Data[col].cat.categories:
#         My_Data[col] = My_Data[col].cat.add_categories(['Unknown'])
#     My_Data[col] = My_Data[col].fillna('Unknown')

# # Processing numerical columns
# for col in ['product_id', 'aisle_id', 'department_id', 'add_to_cart_order', 'reordered']:
#     My_Data[col] = My_Data[col].fillna(-1).astype('int32')

-  Now, let's address the missing values ​​from the column (days_since_prior_order)

In [ ]:
# My_Data['days_since_prior_order'] = My_Data['days_since_prior_order'].fillna(-1)

In [ ]:
# missing_data = My_Data.isnull().sum()
# print(missing_data)

# print("-------------------------------")
# missing_percentage = (My_Data.isnull().sum() / len(My_Data)) * 100
# print(missing_percentage)



---


# **• Distribution plots for numeric features and target(s) (histogram, density).**

### ***-  These six columns were chosen because they encompass the three fundamental dimensions of any recommendation system: time (hour and day of order), behavior (number of orders and basket order), and periodicity (days since the last order). These columns are the most correlated with the target variable (reordered).***

-   **Determine peak activity**

     The diagram shows that activity starts to rise from 7 am and reaches its peak between 10 am and 4 pm.

In [ ]:
# hourly_dist = My_Data['order_hour_of_day'].value_counts().sort_index()

# plt.figure(figsize=(10, 5))
# sns.barplot(x=hourly_dist.index, y=hourly_dist.values, color='teal')
# plt.title('Distribution of Orders by Hour of Day')
# plt.xlabel('Hour of Day')
# plt.ylabel('Total Orders')
# plt.show()
# plt.close()

# del hourly_dist
# gc.collect()

-  **Distribution of days of the week**

    The distribution of orders across the days of the week (from 0 to 6) shows that Saturday and Sunday are the most active days.

In [ ]:
# dow_dist = My_Data['order_dow'].value_counts().sort_index()

# plt.figure(figsize=(8, 4))
# sns.barplot(x=dow_dist.index, y=dow_dist.values, palette='viridis')
# plt.title('Distribution of Orders by Day of Week')
# plt.xticks(ticks=[0,1,2,3,4,5,6], labels=['Sat','Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri'])
# plt.show()
# plt.close()

# del dow_dist
# gc.collect()

-  Distribution of days since the last request

    This graph (Density Plot) shows when customers will return to buy again. We notice two clear peaks; one at 7 days and another very sharp one at 30 days.

In [ ]:
# sample_data = My_Data['days_since_prior_order'].dropna().sample(2000000)

# plt.figure(figsize=(10, 5))
# sns.kdeplot(sample_data, fill=True, color="r")
# plt.title('Density Plot: Days Since Prior Order')
# plt.show()
# plt.close()

-  **Order of adding products to the cart**

    The probability of a product being the first, second, or tenth item in the shopping cart is shown; the vast majority of products are added to the first five positions (1 to 5), and the higher the position, the fewer the number.

    - Because the dataset exceeds 32 million records, plotting the Key Density Curve (KDE) requires immense processing power. I used a random sample of 2 million records to ensure accurate representation of the statistical distribution while maintaining performance efficiency and processing speed.

    - The value (-1) was used as the Sentinel Value to represent initial requests that contained missing values. Having this value appear separately in the Distribution Plot ensures that new user data is not mixed with that of repeat users, and prevents the model from being biased towards average values ​​such as 7 or 8 days.

In [ ]:
# cart_dist = My_Data['add_to_cart_order'].value_counts().head(50)

# plt.figure(figsize=(12, 5))
# sns.barplot(x=cart_dist.index, y=cart_dist.values, color='salmon')
# plt.title('Distribution of Add-to-Cart Order (Top 50)')
# plt.xlabel('Order in Cart')
# plt.ylabel('Count')
# plt.show()
# plt.close()

# del cart_dist
# gc.collect()

-  **Customer loyalty**

    The distribution of the number of requests per user is shown. The distribution is right-skewed, as most users have a small number of requests, while the number gradually decreases for "loyal" users who have requested more than 50 times.

In [ ]:
# plt.figure(figsize=(10, 5))
# plt.hist(My_Data['order_number'], bins=50, color='skyblue', edgecolor='black')
# plt.title('Distribution of Order Number (Customer Loyalty)')
# plt.xlabel('Order Number')
# plt.ylabel('Frequency')
# plt.show()
# plt.close()

-  **Target variable**

    The graph shows the ratio of products that are repurchased versus those that are purchased for the first time.

# **• Categorical cardinality analysis (barplots / top-k frequencies)**

-  We focused on analyzing aisles and sections because they represent structural levels of data. Analyzing them allows us to understand the distribution of purchasing power and identify the categories that dominate transaction volume, which helps simplify the model's feature engineering and ensures its ability to differentiate between fast-moving consumer goods and seasonal goods.

-  **Focus on the most impactful corridors**

    Analysis of all aisles reveals that the Fresh Fruits aisle leads the list with exceptional purchasing volume. This indicates that repurchase decisions in this project are directly influenced by perishable goods, justifying the focus on short-term (7-day) order fulfillment patterns.

In [ ]:
# top_aisles = My_Data['aisle'].value_counts()

# plt.figure(figsize=(15, 30))

# sns.barplot(
#     x=top_aisles.values,
#     y=top_aisles.index,
#     order=top_aisles.index,
#     palette='magma'
# )

# plt.title('Top Frequent Aisles (Sorted from High to Low)')
# plt.xlabel('Count of Items')
# plt.ylabel('Aisle Name')
# plt.show()
# plt.close()

- We moved from detailed aisle analysis to departmental analysis to gain a comprehensive strategic perspective. The results showed that the Produce department represented the largest weight in the data, reinforcing our hypothesis that repurchase behavior is closely linked to fresh, everyday produce.

In [ ]:
# dept_counts = My_Data['department'].value_counts()

# plt.figure(figsize=(12, 8))
# sns.barplot(
#     x=dept_counts.values,
#     y=dept_counts.index,
#     order=dept_counts.index,
#     palette='viridis'
# )

# plt.title('Frequency of Items by Department (Sorted)')
# plt.xlabel('Count of Items')
# plt.ylabel('Department Name')
# plt.show()
# plt.close()

# del dept_counts
# gc.collect()

-  I performed a **categorical cardinality** analysis using a logarithmic scale to understand the diversity of the textual data. This analysis is essential for selecting the appropriate encoding strategy for each variable; it showed that the products column has high cardinality and requires special processing, while sections and corridors represent a balanced classification level from which the model can learn effectively without excessive memory consumption.

In [ ]:
# categorical_cols = ['aisle', 'department', 'product_name', 'eval_set']
# cardinality = My_Data[categorical_cols].nunique().sort_values()

# plt.figure(figsize=(10, 5))
# cardinality.plot(kind='barh', color='orange')
# plt.title('Cardinality of Categorical Features (Number of Unique Values)')
# plt.xlabel('Number of Unique Categories')
# plt.xscale('log')
# plt.show()
# plt.close()

# del cardinality
# gc.collect()

# **Correlation matrix, heatmap and pairwise scatter plots for selected numeric features.**

-  **Correlation Matrix / Heatmap**

    This graph tells us how strong the relationship is between any two variables with a number between (-1 and 1).

In [ ]:
# numeric_cols = ['order_number', 'order_dow', 'order_hour_of_day',
#                 'days_since_prior_order', 'add_to_cart_order', 'reordered']

# corr_matrix = My_Data[numeric_cols].corr()
# plt.figure(figsize=(10, 8))
# sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
# plt.title('Correlation Matrix of Numeric Features')
# plt.show()
# plt.close()

# del corr_matrix
# gc.collect()

-  **Pairwise Scatter Plots**

    A sample of 50,000 records was used because plotting a pairwise array requires extremely intensive computations that memory cannot handle with such a large dataset. Furthermore, plotting millions of points leads to overplotting, which obscures the details of class distribution, whereas a random sample provides an ideal balance between statistical accuracy and visual clarity.

    **What do the three colors mean?**
    
    - Dark color (1): Represents **reordered** products
    - Medium color (0): Represents **new products** purchased for the first time
    - Very light color (-1): Represents the **first orders for new users** (First Orders)

-  I developed the dual-color graph to include three color categories instead of two. This allowed us to clearly distinguish the initial requests (represented by the value -1). Through this graph, we demonstrated that the data from new users is completely isolated and does not cause any interference with the reorder patterns of existing users, thus improving the quality of the data input to the machine learning model.

In [ ]:
# data_sample = My_Data[numeric_cols].sample(50000)
# sns.pairplot(data_sample, hue='reordered', diag_kind='kde', plot_kws={'alpha': 0.5})
# plt.suptitle('Pairwise Scatter Plots (Sample of 50k rows)', y=1.02)
# plt.show()
# plt.close('all')

# del data_sample
# gc.collect()

# **• Time-of-day, day-of-week, and monthly seasonality plots.**

- ##  We analyzed the seasonality of the time period to uncover purchasing cycles. The results showed a significant concentration of orders at the beginning of the week and during daylight hours (10:00 to 16:00). These time patterns provide important context for the model, as peak times are typically associated with 'stocking up' orders, which are characterized by high reorder rates.

- ##  We did not plot monthly seasonality because the Instacart dataset does not provide absolute calendar dates (timestamps), but rather relative time data focusing on the day of the week and hour of the day. Instead, we relied on days_since_prior_order analysis to uncover monthly patterns in repurchase behavior, with the results showing a clear peak at 30 days, representing the customers' monthly shopping cycle.

- **Daily Seasonality**

In [ ]:
# hourly_season = My_Data.groupby('order_hour_of_day')['order_id'].nunique()

# plt.figure(figsize=(12, 5))
# plt.plot(hourly_season.index, hourly_season.values, marker='o', linestyle='-', color='b')
# plt.fill_between(hourly_season.index, hourly_season.values, alpha=0.1, color='b')
# plt.title('Daily Seasonality: Orders by Hour of Day')
# plt.xlabel('Hour (24h Format)')
# plt.ylabel('Total Orders')
# plt.grid(True, axis='y', linestyle='--', alpha=0.7)
# plt.xticks(range(0, 24))
# plt.show()
# plt.close()

- **Weekly Seasonality**

In [ ]:
# weekly_season = My_Data.groupby('order_dow')['order_id'].nunique()

# plt.figure(figsize=(10, 5))
# sns.lineplot(x=weekly_season.index, y=weekly_season.values, marker='s', color='b', linewidth=2.5)
# plt.title('Weekly Seasonality: Orders by Day of Week')
# plt.xticks(ticks=[0,1,2,3,4,5,6], labels=['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat'])
# plt.ylabel('Total Orders')
# plt.grid(True, alpha=0.3)
# plt.show()
# plt.close()

- # *Outlier detection & treatment (winsorizing or removal with justification)*

-  ### We limited the analysis of outliers to these three variables because they represent the quantitative behavioral features of customers. As continuous variables, they are the most susceptible to statistical dispersion, which can skew the predictive model. The remaining columns are either categorical or time-bound, confined to a fixed range, and do not require outlier analysis

-  A comprehensive ****digital audit**** was performed on the entire dataset. Using the ****Interquartile Range (IQR) equation****, we successfully determined the precise number and percentage of outliers for each behavioral variable. This digital audit validated the visual observations in the ****box plots**** .

In [ ]:
# # before
# cols_to_check = ['order_number', 'add_to_cart_order', 'days_since_prior_order']

# print("--- Digital Outlier Detection Report---")

# for col in cols_to_check:
#     Q1 = My_Data[col].quantile(0.25)
#     Q3 = My_Data[col].quantile(0.75)
#     IQR = Q3 - Q1

#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR

#     outliers = My_Data[(My_Data[col] < lower_bound) | (My_Data[col] > upper_bound)]
#     count = outliers.shape[0]
#     percentage = (count / len(My_Data)) * 100

#     print(f"\nColomen: {col}")
#     print(f"Max {upper_bound}")
#     print(f"Number Of outliers: {count:,}")
#     print(f"Percentage of outliers: {percentage:.2f}%")

-  This step was not merely a drawing, but an ****auditing process****; we proved numerically and visually that the outliers were not just random errors, but extreme values ​​representing a significant percentage (up to 5.5%), making the decision to address them using ****Winsorization**** a decision based on strong statistical evidence to ensure the model's stability later on.

In [ ]:
# cols_to_plot = ['order_number', 'add_to_cart_order', 'days_since_prior_order']

# plt.figure(figsize=(20, 10))

# for i, col in enumerate(cols_to_plot, 1):
#     plt.subplot(1, 3, i)
#     sns.boxplot(y=My_Data[col], color='skyblue')
#     plt.title(f'Outliers in {col}')

# plt.tight_layout()
# plt.show()
# plt.close()



# gc.collect()

-  We ruled out the removal option because we are dealing with a massive database, and deleting outliers would result in the loss of over 1.7 million records, potentially preventing the model from understanding the behavior of its most active customers. Instead, we opted for Winsorizing to address statistical outliers by restricting them to the 99th percentile. This decision ensures data integrity and provides a stable training environment for the predictive model without sacrificing sample size.

-  I experimented with the 99th and 95th percentiles and found that 0.95 was the most suitable for our project. This is because Instacart data exhibits very high dispersion in purchasing behavior, and the 95th percentile more closely aligns with the statistical IQR limits we previously calculated. This ensures a reduction in standard deviation and the fitting of more homogeneous features, which helps the algorithm learn more quickly and accurately.

In [ ]:
cols_to_fix = ['order_number', 'add_to_cart_order', 'days_since_prior_order']
for col in cols_to_fix:
    upper_limit = My_Data[col].quantile(0.95)
    My_Data[col] = np.where(My_Data[col] > upper_limit, upper_limit, My_Data[col])

    print(f"Done: {col} | New Max Limit: {upper_limit}")

In [ ]:
# cols_to_plot = ['order_number', 'add_to_cart_order', 'days_since_prior_order']

# plt.figure(figsize=(20, 10))

# for i, col in enumerate(cols_to_plot, 1):
#     plt.subplot(1, 3, i)
#     sns.boxplot(y=My_Data[col], color='lightgreen')
#     plt.title(f'Cleaned Outliers in {col}')

# plt.tight_layout()
# plt.show()
# plt.close('all')

# gc.collect()

- ## Encoding Categorical Variables


In [ ]:
encoding_stats = My_Data[['eval_set','product_name','product_id', 'user_id', 'aisle_id', 'department_id', 'order_dow', 'order_hour_of_day']].nunique()

print("--- Cardinality Report ---")
print(encoding_stats)

- ## **One-Hot Encoding**
  - ***Low-cardinality columns were chosen to ensure that the data dimensions did not become so large as to hinder processing.***
  - ***(eval_set)*** : Convert from one column to two columns (because the original had 3 categories)
  - ***(order_dow)*** : Convert to 6 columns (because the original had 7 categories)
  - ***(department_id)*** : Convert to 21 columns (because the original had 22 categories)
  - ***(order_hour_of_day)*** : Convert to 23 columns (because the original had 24 categories)

In [ ]:
low_card_cols = ['eval_set', 'order_dow', 'department_id', 'order_hour_of_day']
My_Data = pd.get_dummies(My_Data, columns=low_card_cols, drop_first=True, dtype='int8')

cols_to_drop = ['department', 'aisle', 'aisle_id']
My_Data.drop(columns=['department', 'aisle'], inplace=True, errors='ignore')

print("Done! One-Hot Encoding complete.")
print(f"New shape: {My_Data.shape}")

In [ ]:
print( {My_Data.shape})

- ## **Target Encoding**
   -  ***The columns were selected because they contained thousands of unique values ​​that could not be processed using One-Hot Encoding.***
   -  ***(product_id) , (user_id) , (aisle_id)***
   -  The target columns were transformed from random integers to fractional floats, often ranging between 0 and 1. These values ​​accurately represent the "reorder probability" for each product and each user, making the model able to distinguish between "highly requested" and "passing" products with great ease.

- ## **Frequency encoding**
- ***Frequency encoding was chosen to represent the product_name column because it adds a new statistical feature to the model: product popularity. Unlike hashing tricks, which produce random values ​​that may suffer collisions, frequency encoding provides a logical relationship between the number of times a product appears and the probability of it being repurchased, thus improving prediction accuracy while maintaining memory efficiency.***
- Converting long text names into numerical values ​​that express the "weight" or "popularity" of a category helps the model understand patterns associated with bestsellers.

In [ ]:
product_counts = My_Data['product_name'].value_counts()

My_Data['product_name'] = My_Data['product_name'].map(product_counts)
print(My_Data[['product_name']].head())

In [ ]:
print(My_Data.info())

- ## ****Comparison of Encoding Approaches****

  -  **Key Observations:**

    - One-Hot Encoding preserved full interpretability but increased dimensionality (52 additional columns).
    - Target Encoding dramatically reduced dimensions while injecting predictive signal (reorder probability), with smoothing=10 applied to prevent                   overfitting on rare categories.
    - Frequency Encoding provided a simple, meaningful feature (popularity count) without any risk of leakage or dimensionality explosion.

****The combined approach achieved excellent memory efficiency (~3.5 GB total) while introducing strong predictive features, making the dataset suitable for both linear and tree-based models. Target leakage was avoided in Target Encoding by restricting fitting to training folds only within cross-validation pipelines.****

- ## ****Target Encoding – Leakage Prevention****

  - ****Target encoding may introduce data leakage if category statistics are computed using the full dataset.
  To prevent this, **K-fold target encoding** is applied within the training data, where target means
  are computed using K−1 folds and applied to the held-out fold.****
 ****This ensures that each sample is encoded without using its own target value.****


- ###  K-fold target encoding for product_id



In [ ]:
My_Data["product_id_kfold_te"] = np.nan

kf = KFold(n_splits=5, shuffle=True, random_state=42)
global_mean = My_Data["reordered"].mean()

for train_idx, val_idx in kf.split(My_Data):
    train_fold = My_Data.iloc[train_idx]
    val_fold = My_Data.iloc[val_idx]
    means = train_fold.groupby("product_id")["reordered"].mean()
    My_Data.loc[val_idx, "product_id_kfold_te"] = val_fold["product_id"].map(means)

My_Data["product_id_kfold_te"] = My_Data["product_id_kfold_te"].fillna(global_mean)

print("product_id K-fold encoding done!\n")
print(My_Data[["product_id", "product_id_kfold_te"]].head())



- ###  K-fold target encoding for user_id


In [ ]:
My_Data["user_id_kfold_te"] = np.nan

kf = KFold(n_splits=5, shuffle=True, random_state=42)
global_mean = My_Data["reordered"].mean()

for train_idx, val_idx in kf.split(My_Data):
    train_fold = My_Data.iloc[train_idx]
    val_fold = My_Data.iloc[val_idx]
    means = train_fold.groupby("user_id")["reordered"].mean()
    My_Data.loc[val_idx, "user_id_kfold_te"] = val_fold["user_id"].map(means)

My_Data["user_id_kfold_te"] = My_Data["user_id_kfold_te"].fillna(global_mean)

print("user_id K-fold encoding done!\n")
print(My_Data[["user_id", "user_id_kfold_te"]].head())

- ###  K-fold target encoding for aisle_id


In [ ]:
My_Data["aisle_id_kfold_te"] = np.nan

kf = KFold(n_splits=5, shuffle=True, random_state=42)
global_mean = My_Data["reordered"].mean()

for train_idx, val_idx in kf.split(My_Data):
    train_fold = My_Data.iloc[train_idx]
    val_fold = My_Data.iloc[val_idx]
    means = train_fold.groupby("aisle_id")["reordered"].mean()
    My_Data.loc[val_idx, "aisle_id_kfold_te"] = val_fold["aisle_id"].map(means)

My_Data["aisle_id_kfold_te"] = My_Data["aisle_id_kfold_te"].fillna(global_mean)

print("aisle_id K-fold encoding done!\n")
print(My_Data[["aisle_id", "aisle_id_kfold_te"]].head())

### Observations on K-fold Target Encoding Results

When comparing the K-fold encoded values to the original global Target Encoding:

- For `product_id` and `aisle_id`, the differences between global and K-fold means were very small.
- For `user_id`, the differences were noticeably larger.



- ## ****Feature Scaling****

   - Feature scaling was performed to ensure numerical features contribute fairly to           models sensitive to magnitude.

- **StandardScaler** was chosen (mean=0, std=1) as it is robust to outliers and suitable for distance-based models (KNN, SVM).
- Applied to numerical features: `order_number`, `days_since_prior_order`, `add_to_cart_order`.

In [ ]:
numeric_cols = ['order_number', 'days_since_prior_order', 'add_to_cart_order']

My_Data[col] = My_Data[col].replace([np.inf, -np.inf], np.nan).fillna(My_Data[col].max())
scaler = StandardScaler()

My_Data[numeric_cols] = scaler.fit_transform(My_Data[numeric_cols])
print("Scaling done on:", numeric_cols , "\n")
print(My_Data[numeric_cols].describe().round(2))

- # ****Feature Engineering****


- ## ****User-level features****

User-level features were engineered by aggregating data per `user_id` to capture individual customer behavior:

- **Total number of orders**: Count of unique orders per user.
- **Average basket size**: Mean number of items per order.
- **Reorder ratio**: Proportion of reordered items across all orders (total reordered items / total items purchased).
- **Mean days between orders**: Average `days_since_prior_order` for the user.
- **Last order recency**: Number of days since the user's most recent order (relative to the latest date in the dataset or prediction point).

These features were computed using `groupby('user_id')` on the orders and order_products tables, then merged back to the main dataset.


### Total number of orders per user



In [ ]:
user_total_orders = orders.groupby('user_id')['order_id'].nunique().rename('user_total_orders').reset_index()

print("Total number of orders per user" , "\n")
print(user_total_orders.head())

### Average basket size per user


The merge between `orders` and `basket_sizes` on `order_id` was necessary to attach the basket size (number of items per order) to each order record in the `orders` DataFrame.

- `basket_sizes` was derived from `order_products_prior` using groupby on `order_id` to count items per order.
- The `orders` DataFrame contains user and order metadata (including `user_id`) but does not include basket size information.
- Merging on `order_id` (left join) added the `basket_size` column to each order, enabling subsequent groupby on `user_id` to compute the **average basket size per user**.



In [ ]:
basket_sizes = order_products_prior.groupby('order_id')['product_id'].count().rename('basket_size').reset_index()
orders_with_basket = orders.merge(basket_sizes, on='order_id', how='left')
user_avg_basket = orders_with_basket.groupby('user_id')['basket_size'].mean().rename('user_avg_basket_size').reset_index()

print("Average basket size per user" , "\n")
print(user_avg_basket.head())

### Reorder ratio per user

In [ ]:
order_to_user = orders.set_index('order_id')['user_id'].to_dict()
order_products_prior['user_id'] = order_products_prior['order_id'].map(order_to_user)
user_reorder = order_products_prior.groupby('user_id')['reordered'].mean().rename('user_reorder_ratio').reset_index()

print("Reorder ratio per user" , "\n")
print(user_reorder.head())

### Mean days between orders per user

In [ ]:
user_mean_days = orders.groupby('user_id')['days_since_prior_order'].mean().rename('user_mean_days_between_orders').reset_index()

print("Mean days between orders per user" , "\n")
print(user_mean_days.head())

### Last order recency per user

In [ ]:
max_order_num = orders.groupby('user_id')['order_number'].max().reset_index()

last_orders = max_order_num.merge(orders, on=['user_id', 'order_number'], how='left')

last_orders['user_last_order_recency'] = last_orders['days_since_prior_order'].fillna(0)

user_recency = last_orders[['user_id', 'user_last_order_recency']].drop_duplicates()

print("Last order recency per user" , "\n")
print(user_recency.head())

### Merged for all user-level features

In [ ]:
My_Data['user_id'] = My_Data['user_id'].astype('int32')

user_tables = [user_total_orders, user_avg_basket, user_reorder, user_mean_days, user_recency]

for df in user_tables:
    if 'user_id' in df.columns:
        df['user_id'] = df['user_id'].astype('int32')

    current_float_cols = df.select_dtypes(include=['float64']).columns
    df[current_float_cols] = df[current_float_cols].astype('float32')

merge_all_features = user_total_orders \
    .merge(user_avg_basket, on='user_id') \
    .merge(user_reorder, on='user_id') \
    .merge(user_mean_days, on='user_id') \
    .merge(user_recency, on='user_id')

My_Data = My_Data.merge(merge_all_features, on='user_id', how='left')

del merge_all_features, user_total_orders, user_avg_basket, user_reorder, user_mean_days, user_recency
gc.collect()

print(My_Data.shape)

In [ ]:
print(My_Data.columns)
print("Final shape:", My_Data.shape)

- ## ****Product-level features****


 - ### ****product_reorder_rate****

In [ ]:
# Calculate reorder statistics for each product
product_features = order_products_prior.groupby('product_id')['reordered'].agg(['mean', 'count']).reset_index()
product_features.columns = ['product_id', 'product_reorder_rate', 'product_total_purchases']

# Convert data to float32 to save RAM
product_features['product_reorder_rate'] = product_features['product_reorder_rate'].astype('float32')
product_features['product_total_purchases'] = product_features['product_total_purchases'].astype('int32')

print(product_features.head())

 - ### ****avg_pos_in_cart****

In [ ]:
# Calculating the average product ranking in the cart
avg_pos = order_products_prior.groupby('product_id')['add_to_cart_order'].mean().reset_index()
avg_pos.columns = ['product_id', 'avg_pos_in_cart']

# Convert to float32
avg_pos['avg_pos_in_cart'] = avg_pos['avg_pos_in_cart'].astype('float32')

# Integrate it with the product features table
product_features = product_features.merge(avg_pos, on='product_id', how='left')

del avg_pos
gc.collect()
print(product_features.head())

 - ### ****popularity_over_time****

In [ ]:
# Temporary merge to get the order time for each product
product_time = order_products_prior[['order_id', 'product_id']].merge(orders[['order_id', 'order_hour_of_day']], on='order_id', how='left')

# Calculating the average purchase time for each product
avg_hour = product_time.groupby('product_id')['order_hour_of_day'].mean().reset_index()
avg_hour.columns = ['product_id', 'product_avg_hour_of_day']

# Convert to float32 and final merge
avg_hour['product_avg_hour_of_day'] = avg_hour['product_avg_hour_of_day'].astype('float32')
product_features = product_features.merge(avg_hour, on='product_id', how='left')

# Memory Cleansing
del product_time, avg_hour
gc.collect()
print(product_features.head())

 - ### ****Final integration with (My_Data)****

In [ ]:
# Integrating product features with the main table
My_Data = My_Data.merge(product_features, on='product_id', how='left')

# Delete the intermediate table to save space
del product_features
gc.collect()
print(My_Data.shape)

In [ ]:
print(My_Data.shape)
print(My_Data.head())

- ## ****User×Product interaction features :****


 - ### **up_purchase_count**

In [ ]:
# Calculate the number of times each user purchases each product
up_features = order_products_prior.groupby(['user_id', 'product_id'])['order_id'].count().reset_index()
up_features.columns = ['user_id', 'product_id', 'up_purchase_count']

# Convert to int32 to save RAM
up_features['up_purchase_count'] = up_features['up_purchase_count'].astype('int32')

print(up_features.head())

 - ### **up_days_since_last_purchase**

In [ ]:
# Merge the products table with the orders table to get order_number
# We only need two columns from the orders table to save RAM.
temp_orders = orders[['order_id', 'order_number']]
order_products_prior = order_products_prior.merge(temp_orders, on='order_id', how='left')

# Account last order number where the user purchased this product
last_order_per_up = order_products_prior.groupby(['user_id', 'product_id'])['order_number'].max().reset_index()

# Get the time (how many days have passed)
up_last_order = last_order_per_up.merge(orders[['user_id', 'order_number', 'days_since_prior_order']],
                                        on=['user_id', 'order_number'], how='left')

# Rename the column to make it clear
up_last_order.rename(columns={'days_since_prior_order': 'up_days_since_last_purchase'}, inplace=True)

# Integrate the feature with the up_features table
up_features = up_features.merge(up_last_order[['user_id', 'product_id', 'up_days_since_last_purchase']],
                                 on=['user_id', 'product_id'], how='left')

# Clean the RAM immediately
del temp_orders, last_order_per_up, up_last_order
gc.collect()

print(up_features.head())

 - ### **up_reorder_probability**

In [ ]:
user_orders_count = My_Data[['user_id', 'user_total_orders']].drop_duplicates()

up_stats = order_products_prior.groupby(['user_id', 'product_id'])['order_number'].agg(['min', 'max']).reset_index()
up_stats.columns = ['user_id', 'product_id', 'first_order_number', 'last_order_number']

up_counts = order_products_prior.groupby(['user_id', 'product_id'])['order_id'].count().reset_index()
up_counts.columns = ['user_id', 'product_id', 'up_purchase_count']

up_features = up_counts.merge(up_stats, on=['user_id', 'product_id'], how='left')
up_features = up_features.merge(user_orders_count, on='user_id', how='left')

up_features['up_reorder_probability'] = up_features['up_purchase_count'] / \
                                         (up_features['user_total_orders'] - up_features['first_order_number'] + 1)

up_features = up_features.merge(orders[['user_id', 'order_number', 'days_since_prior_order']],
                                 left_on=['user_id', 'last_order_number'],
                                 right_on=['user_id', 'order_number'], how='left')

up_features.rename(columns={'days_since_prior_order': 'up_days_since_last_purchase'}, inplace=True)

up_features['up_reorder_probability'] = up_features['up_reorder_probability'].astype('float32')
up_features['up_days_since_last_purchase'] = up_features['up_days_since_last_purchase'].fillna(0).astype('float32')

up_features.drop(['first_order_number', 'last_order_number', 'order_number', 'user_total_orders'], axis=1, inplace=True)

del up_stats, up_counts, user_orders_count
gc.collect()
print(up_features.columns.tolist())
print(up_features.head())

 - ### **Final integration with My_Data**

In [ ]:
My_Data['user_id'] = My_Data['user_id'].astype('float32')
My_Data['product_id'] = My_Data['product_id'].astype('float32')
up_features['user_id'] = up_features['user_id'].astype('float32')
up_features['product_id'] = up_features['product_id'].astype('float32')

My_Data = My_Data.merge(up_features, on=['user_id', 'product_id'], how='left')

del up_features
gc.collect()

print(My_Data.shape)
print(My_Data.columns.tolist())

- ## ****Temporal features: hour/day/month/year, season, holiday flags (if available)****
  - Since we do not have a complete history (year/month) in the core Instacart dataset, we will focus on the available features (hour and day) and build “flags” from them that represent time periods and seasons.

- ## **Implemented Features**

- ****All available time data within the original files has been exhausted. The current feature-engineered data is perfectly sufficient to model user behavior based on daily and weekly patterns.****

In [ ]:
# Extracting available raw time data from the command table
# We use drop_duplicates to ensure that rows are not duplicated when merging
temporal_info = orders[['order_id', 'order_hour_of_day', 'order_dow', 'days_since_prior_order']].drop_duplicates()

In [ ]:
# Standardizing data types to ensure fast and warning-free integration.
temporal_info['order_id'] = temporal_info['order_id'].astype('int32')
My_Data['order_id'] = My_Data['order_id'].astype('int32')

In [ ]:
# Merging Time Features with the My_Data Main Table
# We only select columns that have not been merged previously to avoid duplication
cols_to_merge = temporal_info.columns.difference(My_Data.columns).tolist() + ['order_id']
My_Data = My_Data.merge(temporal_info[cols_to_merge], on='order_id', how='left')

   - ### ****Hour of Day****

In [ ]:
# Build the "Day Periods" feature based on the available on-demand clock.
def get_day_period(hour):
    if 5 <= hour < 12: return 1    # Morning
    elif 12 <= hour < 17: return 2  # Afternoon
    elif 17 <= hour < 21: return 3  # Evening
    else: return 4                 # Night

My_Data['day_period'] = My_Data['order_hour_of_day'].apply(get_day_period).astype('int8')


- ### ****Day of the Week****

In [ ]:
# Build a "Weekend Flag" feature based on the available day number
# In Instacart: Days 0 and 1 represent peak demand (often Saturday and Sunday)
My_Data['is_weekend'] = My_Data['order_dow'].apply(lambda x: 1 if x in [0, 1] else 0).astype('int8')

In [ ]:
# 6. Addressing the "Days Since Last Order" feature that is already available
My_Data['days_since_prior_order'] = My_Data['days_since_prior_order'].fillna(0).astype('float32')
# Clean the memory immediately after completion
del temporal_info
gc.collect()
print(['day_period', 'is_weekend', 'days_since_prior_order'])

- ## ****Calculating the features of the "last 3 orders" for each user and product****
  - #### ****We will know how many times the user purchased this product in their last 3 visits to the store****

In [ ]:
# Determine the last order number for each user (Max Order Number)
user_last_orders = orders.groupby('user_id')['order_number'].max().reset_index()
user_last_orders.columns = ['user_id', 'max_order_number']

# Create the missing variable by combining product data with maximum order numbers
# This is the line you were missing, which was causing the NameError
last_3_orders_data = order_products_prior.merge(user_last_orders, on='user_id', how='left')

# Filter data to only the last 3 requests
last_3_orders_data = last_3_orders_data[last_3_orders_data['order_number'] > (last_3_orders_data['max_order_number'] - 3)]

# Calculate the number of purchases within this time window
up_last_3_count = last_3_orders_data.groupby(['user_id', 'product_id'])['order_id'].count().reset_index()
up_last_3_count.columns = ['user_id', 'product_id', 'up_last_3_purchase_count']

# Final integration with the My_Data table
My_Data = My_Data.merge(up_last_3_count, on=['user_id', 'product_id'], how='left')
My_Data['up_last_3_purchase_count'] = My_Data['up_last_3_purchase_count'].fillna(0).astype('int8')

# Memory cleaning
del last_3_orders_data, up_last_3_count, user_last_orders
gc.collect()

print(My_Data[['user_id', 'product_id', 'up_last_3_purchase_count']].head())

- ## ****Calculating the "Last Order Rate" feature****
  - #### ****This is a very powerful feature that tells the model if the product is in the last basket purchased by the user.****

In [ ]:
# Pre-cleaning: Drop any existing versions of the column to avoid duplication errors
cols_to_drop = [c for c in My_Data.columns if 'is_in_last_order' in c]
if cols_to_drop:
    My_Data.drop(columns=cols_to_drop, inplace=True)

#  Limit the last order to each user.
user_last_order = orders.groupby('user_id')['order_number'].max().reset_index()
user_last_order.columns = ['user_id', 'order_number']

# Knowing which products were in that order
last_order_products = order_products_prior.merge(user_last_order, on=['user_id', 'order_number'], how='inner')
last_order_products['is_in_last_order'] = 1

# Integration with My_Data
My_Data = My_Data.merge(last_order_products[['user_id', 'product_id', 'is_in_last_order']],
                        on=['user_id', 'product_id'], how='left')

My_Data['is_in_last_order'] = My_Data['is_in_last_order'].fillna(0).astype('int8')
My_Data[col] = My_Data[col].replace([np.inf, -np.inf], np.nan).fillna(0).astype('int8')

del user_last_order, last_order_products
gc.collect()

print(My_Data.head())

- ## ****Log Transform****

In [ ]:
# Converting the total product purchases to a logarithmic scale
# We use +1 to avoid the zero logarithm
My_Data['log_total_product_sales'] = np.log1p(My_Data['product_total_purchases']).astype('float32')
print(My_Data.head())
gc.collect()

### We have engineered 83 features categorized into 5 strategic groups to drive model performance:

 Identifiers: user_id, product_id, order_id (Linking keys).
 User Profile: total_orders, avg_basket, reorder_ratio (Customer history & loyalty).
 Product Stats: reorder_rate, popularity_score (Item characteristics).
 Interaction: up_purchase_count, up_prob, in_last_3_orders (User-specific habits).
 Context: hour, day, weekend_flag (Temporal patterns).

In [ ]:
print(len(My_Data.columns))
print(My_Data.columns.tolist())

### Final Data Cleaning: Addressing Residual NaNs
Upon final inspection, 206,209 rows (< 1% of data) contained NaN values in product-specific features.

Audit Findings: These rows correspond to the Train/Test set entries (from orders.csv) which represent future orders. By definition, these new orders do not yet have product history directly attached via the prior dataset merge logic.

Action Plan: We cannot drop these rows as they are the target inputs for our model. instead, we apply a domain-aware imputation strategy:

Missing Counts/Rates: Fill with 0. (Implies no history exists yet for this specific context).
Result: Dataset integrity is preserved at 100%, ensuring every row is valid for prediction.

In [ ]:
nan_counts = My_Data.isnull().sum()
print(nan_counts[nan_counts > 0])

### Final Data Sanitization: Handling Residual Nulls
After merging all engineered features, we perform a final audit for missing values.

Observation: We expect to find NaN values in product-interaction features (e.g., up_purchase_count, product_reorder_rate) specifically for the Train/Test set rows.

Reason: These rows represent new orders where the specific user-product interaction might not have occurred in the prior history, or the product features could not be mapped (cold start).

In [ ]:
nan_summary = My_Data.isnull().sum()
nan_only = nan_summary[nan_summary > 0]
print(nan_only)

if len(nan_only) > 0:
    cols_to_fix = nan_only.index.tolist()

    My_Data[cols_to_fix] = My_Data[cols_to_fix].fillna(0)

print(len(nan_only))

### Dimensionality & collinearity
We will now perform a Variance Inflation Factor (VIF) test to identify and remove features with severe multicollinearity ($VIF > 10$).

Process:

Sample the Data: Calculating VIF on 32M rows is computationally expensive, so we take a representative random sample of 100,000 rows.

Calculate VIF: For each continuous numerical feature, we compute how much its variance is inflated by the other features.

Action: Features with high VIF scores (indicating they are redundant copies of others) will be candidates for removal to streamline the model.

In [ ]:
exclude_cols = ['user_id', 'product_id', 'order_id', 'reordered', 'eval_set_test', 'eval_set_train']

exclude_cols += [col for col in My_Data.columns if '_kfold_te' in col or 'order_dow_' in col or 'department_id_' in col or 'order_hour_' in col]


numeric_cols = My_Data.select_dtypes(include=['float32', 'float64', 'int32', 'int64', 'int8']).columns.tolist()

features_for_vif = list(set(numeric_cols) - set(exclude_cols))

print(f"Features selected for VIF: {len(features_for_vif)}")
print(features_for_vif)


vif_data = My_Data[features_for_vif].sample(100000, random_state=42).copy()

vif_data = vif_data.replace([np.inf, -np.inf], 0).fillna(0)

vif_df = pd.DataFrame()
vif_df["feature"] = vif_data.columns
vif_df["VIF"] = [variance_inflation_factor(vif_data.values, i)
                 for i in range(len(vif_data.columns))]

print(vif_df.sort_values(by="VIF", ascending=False))

#### Based on the audit results, we detected severe multicollinearity in specific feature groups. To improve model stability and training speed, we are removing the redundant variables


In [ ]:
features_to_drop = [
    'product_avg_hour_of_day',
    'product_total_purchases',
    'product_name',
    'avg_pos_in_cart'
]

cols_present = [c for c in features_to_drop if c in My_Data.columns]

if cols_present:
    My_Data.drop(columns=cols_present, inplace=True)
    print(f"Successfully dropped high VIF features: {cols_present}")
    print(f"New column count: {len(My_Data.columns)}")
else:
    print("Columns already dropped or not found.")


gc.collect()

- # ****Imbalanced data handling (classification)****


In [ ]:
# Reading the train schedule
train_orders = pd.read_csv('/kaggle/input/order_products__train.csv', usecols=['order_id', 'product_id', 'reordered'])

#  Since we want to predict for each (user + product)
# We will use 'product_id' and 'user_id' for linking (you need to link the train to the orders table first to get the user_id)
orders = pd.read_csv('/kaggle/input/orders.csv', usecols=['order_id', 'user_id', 'eval_set'])
train_full = train_orders.merge(orders[['order_id', 'user_id']], on='order_id', how='left')

# Any record present in My_Data but not in train_full will be assigned NaN and then converted to 0
My_Data = My_Data.merge(train_full[['user_id', 'product_id', 'reordered']],
                        on=['user_id', 'product_id'],
                        how='left',
                        suffixes=('', '_final_target'))

# Dr. Cleaning the new target column
My_Data['target'] = My_Data['reordered_final_target'].fillna(0).astype('int8')
My_Data.drop(['reordered_final_target'], axis=1, inplace=True)

In [ ]:
del train_orders
del orders
import gc
gc.collect()

- ### ****Examine the Target (Class Distribution) distribution****

In [ ]:
print(My_Data['target'].value_counts())
print(My_Data['target'].value_counts(normalize=True))

- ### ****Row Count Check****

In [ ]:
print(len(My_Data))
print(len(My_Data))

- ### ****Nulls Check****

In [ ]:
print(My_Data['target'].isnull().sum())

- ### ****Logic Spot-check****

In [ ]:
print(My_Data[My_Data['target'] == 1][['user_id', 'product_id', 'target']].head())

- #### Analysis of the target variable revealed a class imbalance of approximately ****1:7****. This means the data contains seven times more negative samples than positive ones, justifying the need to use imbalance correction techniques to ensure the model's accuracy in predicting buybacks

## $$\frac{\text{Zeros}}{\text{Ones}} = \frac{28,402,695}{4,238,003} \approx 6.7019$$

-----------------------------------------------------------------------------------------------------------
- ## ****Aggregation Strategy****

  - ### We will use an ****Aggregation Strategy**** instead of dealing with the entire dataset,because:
    -  *Transitioning from the order level to the (user-product) level*
    -  *Noise Reduction*
    -  *Computational Efficiency*
    -  *Feature Signal*

In [ ]:
unique_pairs = My_Data.groupby(['user_id', 'product_id']).ngroups
print(f"Number of uniques:{unique_pairs}")

- ### *By implementing an aggregation strategy, we successfully reduced the data size by approximately 58% (from 32.6 million to 13.5 million records) while preserving all user behavioral information. This not only improved memory efficiency but also made imbalance handling more precise and focused on the unique user-product binaries*

In [ ]:
My_Data_Aggregated = My_Data.groupby(['user_id', 'product_id']).last().reset_index()


# Memory Downcasting: Reducing numeric precision to save space
# Converting float64 to float32 and int64 to int32 reduces RAM usage by ~50%
# without significant loss in model accuracy.
for col in My_Data_Aggregated.columns:
    if My_Data_Aggregated[col].dtype == 'float64':
        My_Data_Aggregated[col] = My_Data_Aggregated[col].astype('float32')
    if My_Data_Aggregated[col].dtype == 'int64':
        My_Data_Aggregated[col] = My_Data_Aggregated[col].astype('int32')

# 4. Immediate Garbage Collection: Deleting the large raw dataframe
# and triggering Python's memory manager to free up physical RAM.
del My_Data
gc.collect()

In [ ]:
print( My_Data_Aggregated.shape)
print("-" * 30)
print(My_Data_Aggregated['target'].value_counts(normalize=True))

- ### *After the aggregation process, we observed a decrease in the percentage of positive categories to 6.1%. This reflects the actual nature of the problem; we are moving from predicting at the level of 'every historical purchase' to predicting at the level of 'product repurchase opportunity'. This makes the task more challenging and more strongly justifies the need for the imbalance handling techniques that we will now apply*

## $$\frac{\text{Zeros}}{\text{Ones}} = \frac{0.93867}{0.06133} \approx 15.3$$

---------------------------------------------------------------------------------------------------------------
- ## ****Starting the experimentation process****

- ### ****Class Weights****

In [ ]:
# # Weight calculation (approximately 15.3)
# pos_weight = 0.93867 / 0.06133
# model_class_weights = XGBClassifier(scale_pos_weight=pos_weight, n_estimators=100, tree_method='hist')

- ### ****Undersampling****

In [ ]:
# # Separating the two categories
# minority = My_Data_Aggregated[My_Data_Aggregated['target'] == 1]
# majority = My_Data_Aggregated[My_Data_Aggregated['target'] == 0]

# # Taking a random sample from the majority equal to the minority
# majority_under = majority.sample(len(minority), random_state=42)

# # Merge them into a new, separate trial schedule
# df_under = pd.concat([minority, majority_under])

# print("Undersampling:")
# print(df_under['target'].value_counts(normalize=True))

- ### ****SMOTE****

In [ ]:
# # We're taking a sample for testing to prevent the RAM from exploding
# df_sample_smote = My_Data_Aggregated.sample(1000000, random_state=42)
# X_sample = df_sample_smote.drop('target', axis=1)
# y_sample = df_sample_smote['target']

# sm = SMOTE(random_state=42)
# X_res, y_res = sm.fit_resample(X_sample, y_sample)

# print("SMOTE:")
# print(y_res.value_counts(normalize=True))

- ### Because of a significant class imbalance in the data, we adopted the ****F1-Score**** as the primary criterion for evaluating our experiments, as it provides a precise balance between the ability to detect recalls and the accuracy of those predictions, thus preventing the misleading effects that can be caused by the traditional accuracy metric

In [ ]:
# def get_results(model, X, y, title):
#     model.fit(X, y)
#     preds = model.predict(X)
#     print(f"--- {title} ---")
#     print(f"F1-Score: {f1_score(y, preds):.4f}")
#     print(f"Recall: {recall_score(y, preds):.4f}")
#     print(f"Precision: {precision_score(y, preds):.4f}\n")


# # A slightly modified function for SMOTE because the data is already ready and segmented
# def get_results_for_smote(model, X, y, title):
#     model.fit(X, y)
#     preds = model.predict(X)
#     print(f"--- {title} ---")
#     print(f"F1-Score: {f1_score(y, preds):.4f}")
#     print(f"Recall: {recall_score(y, preds):.4f}")
#     print(f"Precision: {precision_score(y, preds):.4f}\n")


# # Unprocessed Experiment (Original Data - Sample)
# sample_orig = My_Data_Aggregated.sample(100000)
# get_results(XGBClassifier(), sample_orig.drop('target', axis=1), sample_orig['target'], "Baseline")

# # SMOTE
# get_results_for_smote(XGBClassifier(), X_res, y_res, "SMOTE")

# # Class Weights Test
# get_results(XGBClassifier(scale_pos_weight=15), sample_orig.drop('target', axis=1), sample_orig['target'], "Class Weights")

# #3. The Undersampling Experience
# get_results(XGBClassifier(), df_under.drop('target', axis=1), df_under['target'], "Undersampling")

- ## ****Analyze Numbers****

- ### *Baseline (unprocessed): The ٌ****Recall**** is catastrophic (0.22), meaning the model failed to detect 80% of repurchase transactions. It's only "cautious" (high precision) but "afraid" to predict 1*
- ----------------------------------------------------------------------------------------------------------
- ### *SMOTE: The numbers are very perfect (0.93) because we tested it on a small, artificially balanced sample, and it illustrates the "maximum potential" if huge computing resources were available*
- - ----------------------------------------------------------------------------------------------------------
- ### *Class Weights: The Recall jumped to (0.93)! The model became "very bold" in unit hunting, but at the expense of precision, which decreased. This is exactly what was expected when balancing the weights*
- - ----------------------------------------------------------------------------------------------------------
- ### *Undersampling: It gave you the best realistic balance; Precision and Recall are close (0.74 - 0.76), and F1-Score is very high (0.75)*

- ## ****Pros & Cons****
----------------------------------------------------------------------------------------------------------

- ## ****Class Weights****

  - ### ****Pros****: *We haven't lost a single record from the 13 million rows*
  - ### ****Cons****: *This led to a significant increase in False Positives (predicting that the customer will buy when they will not)*

- ## ****Undersampling****

  - ### ****Pros****: *Incredible speed in training and a very balanced performance between hunting single targets and accurate prediction.*
  - ### ****Cons****: *We removed millions of records from category (0), which may prevent the model from learning some special cases.*

- ## ****SMOTE****

  - ### ****Pros****: *It solves the problem of data scarcity by generating intelligent samples instead of simply iterating.*
  - ### ****Cons****: *High memory consumption and cannot be applied to the entire Instacart data (13M) in a limited Kagel environment*

---------------------------------------------------------------------------------------------------------------------
- # ****Time-aware splitting****

- ## Since purchasing behavior is chronological, we avoided random splitting, which could lead to ****(Data Leakage)**** Instead, we adopted a ****(User-Based Splitting)**** strategy, where the model is trained on one group of users and its predictive ability is tested on a completely different group. This ensures the model can ****(Generalize)**** and accurately predict future orders.

- ## Training on raw ****(Transactional Data)**** leads to duplicate features for the same ****(User-Product)****, causing data inflation without adding new predictive information. Instead, we performed ****feature engineering**** to transform the data to the ****(User-Product level)****. This doesn't reduce data quality; rather, it focuses it into ****(Behavioral Features)**** that describe the user's relationship with the product over time—the optimal level for predictive repurchase decisions

In [ ]:
#1. Obtain a list of unique users
all_users = My_Data_Aggregated['user_id'].unique()

In [ ]:
# Randomly mix users once (to ensure fair distribution) with the seed installed
np.random.seed(42)
np.random.shuffle(all_users)

In [ ]:
# Identifying split points
train_end = int(0.8 * len(all_users))
val_end = int(0.9 * len(all_users))

In [ ]:
# Segmenting the user list
train_users = all_users[:train_end]
val_users = all_users[train_end:val_end]
test_users = all_users[val_end:]

In [ ]:
# Extracting actual data based on users
train_df = My_Data_Aggregated[My_Data_Aggregated['user_id'].isin(train_users)]
val_df = My_Data_Aggregated[My_Data_Aggregated['user_id'].isin(val_users)]
test_df = My_Data_Aggregated[My_Data_Aggregated['user_id'].isin(test_users)]

In [ ]:
print(len(My_Data_Aggregated))
print("-"*20)
print(f"{len(train_df)} {len(train_users)}")
print(f"{len(val_df)} {len(val_users)}")
print(f"{len(test_df)} {len(test_users)}")

### Adhering to the principle of (Time-aware Splitting) and to avoid (Data leakage), we split the data based on (User IDs) rather than randomly splitting rows. This ensures that the model is trained on a stable set of users and tested on a completely independent set, mimicking real-world scenarios where the system needs to predict (future orders) based on aggregated  (past history) , without any time overlap between the training and test sets.

In [ ]:
# Examining the link between features and the goal
correlation = train_df.corr(numeric_only=True)['target'].sort_values(ascending=False)
print(correlation.head())

### Preprocessing Completed
We have successfully completed the full preprocessing study as outlined in the project requirements.

### Regression Task: Target Selection

Target Variable: 'days_since_prior_order'

"We chose to predict the time interval between orders rather than reorder counts for its high strategic value. Predicting when a customer will return allows for:

Precision Retargeting: Sending marketing incentives at the exact moment a customer is likely to restock.

Dynamic Scheduling: Optimizing logistics and supply chain operations based on predicted temporal demand.

Customer Churn Prevention: Identifying deviations from predicted ordering cycles to intervene before a customer stops using the platform."

### Feature Isolation:
I isolated behavioral features from identifiers to specifically target the days_since_prior_order prediction.

In [ ]:
# --- Feature Selection & Data Cleaning ---
# Removing identifiers and the classification target to isolate features for regression.
features = [col for col in My_Data_Aggregated.columns if col not in ['user_id', 'product_id', 'target', 'days_since_prior_order']]

X = My_Data_Aggregated[features].copy()

# Handling Categorical columns to prevent "TypeError" during fillna(0).
# We convert categories to strings so that they can accept the new '0' value for missing data.
for col in X.select_dtypes(include=['category']).columns:
    X[col] = X[col].astype(str)

# Filling missing values with 0 for both features and the target variable.
X = X.fillna(0)
y = My_Data_Aggregated['days_since_prior_order'].fillna(0)

# Splitting the dataset into Training (80%) and Testing (20%) sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# RAM Optimization
del X
gc.collect()

print(f"Training set size: {X_train.shape}")
print(f"Testing set size: {X_test.shape}")

### Evaluation Metrics

In [ ]:
def evaluate_regression(name, y_true, y_pred, X_data):

    # 1. Basic Metrics (MAE, MSE, RMSE)
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)

    # 2. R2 and Adjusted R2
    r2 = r2_score(y_true, y_pred)

    # Formula for Adjusted R2
    n = len(y_true)
    k = X_data.shape[1]
    adj_r2 = 1 - ((1 - r2) * (n - 1) / (n - k - 1))

    print(f"--- {name} Performance ---")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R2 Score: {r2:.4f}")
    print(f"Adjusted R2: {adj_r2:.4f}")

    # 3. Heteroscedasticity Test (Breusch-Pagan)
    # Residuals calculation
    residuals = y_true - y_pred

    # Simple Breusch-Pagan test
    print("\n--- Statistical Tests ---")

    bp_test = sms.het_breuschpagan(residuals, sm.add_constant(X_data))
    print(f"Breusch-Pagan p-value: {bp_test[1]:.4f}")

    # 4. Visualizations (Residual Plot & Q-Q Plot)
    plt.figure(figsize=(12, 5))

    # Residual Plot
    plt.subplot(1, 2, 1)
    sns.scatterplot(x=y_pred, y=residuals, alpha=0.5)
    plt.axhline(y=0, color='red', linestyle='--')
    plt.title(f'Residuals vs Predicted ({name})')
    plt.xlabel('Predicted Values')
    plt.ylabel('Residuals')

    # Q-Q Plot
    plt.subplot(1, 2, 2)
    stats.probplot(residuals, dist="norm", plot=plt)
    plt.title(f'Normal Q-Q Plot ({name})')

    plt.tight_layout()
    plt.show()
    plt.close()

### Methodology:
 **Baseline Linear Modeling (OLS, Lasso, Ridge)**
In this stage, we implement three variations of linear regression to establish a performance baseline:
1. **OLS:** Standard linear regression without regularization.
2. **Ridge (L2):** Adds a penalty to prevent large coefficients and reduce model complexity.
3. **Lasso (L1):** Performs automatic feature selection by shrinking less important coefficients to zero.

*Note: We focus the residual analysis on the OLS model as it serves as our primary linear baseline. Its error patterns provide the best diagnostic insight into whether our data satisfies the assumptions required for linear modeling.*

In [ ]:
# --- Pre-processing for Linear Models ---
# Linear models cannot handle strings. We must convert categorical strings into numeric values.
# We use get_dummies to transform categorical columns into a numeric format (One-Hot Encoding).
X_train_numeric = pd.get_dummies(X_train, drop_first=True)
X_test_numeric = pd.get_dummies(X_test, drop_first=True)

# Ensure both train and test have the same columns after encoding
X_train_numeric, X_test_numeric = X_train_numeric.align(X_test_numeric, join='left', axis=1, fill_value=0)

# We only keep features that are numbers (float or int)
# 'eval_set', 'user_id', and 'product_id' must be removed
X_train_numeric = X_train.select_dtypes(include=[np.number])
X_test_numeric = X_test.select_dtypes(include=[np.number])

In [ ]:
# --- 1. Ordinary Least Squares (OLS) ---
# Establishing a performance benchmark with basic linear regression.
ols_reg = LinearRegression()
ols_reg.fit(X_train_numeric, y_train)
ols_pred = ols_reg.predict(X_test_numeric)
evaluate_regression("OLS Regression", y_test, ols_pred, X_test_numeric)

In [ ]:
# --- 2. Ridge Regression (L2 Regularization) ---
# Shrinking coefficients to manage multicollinearity and prevent overfitting.
ridge = Ridge(alpha=1.0)
ridge.fit(X_train_numeric, y_train)
ridge_pred = ridge.predict(X_test_numeric)
evaluate_regression("Ridge (L2)", y_test, ridge_pred, X_test_numeric)

In [ ]:
# --- 3. Lasso Regression (L1 Regularization) ---
# Using L1 penalty for automatic feature selection by zeroing out insignificant weights.
lasso = Lasso(alpha=0.1)
lasso.fit(X_train_numeric, y_train)
lasso_pred = lasso.predict(X_test_numeric)
evaluate_regression("Lasso (L1)", y_test, lasso_pred, X_test_numeric)

In [ ]:
# --- Feature Scaling (Crucial for Elastic Net Regularization) ---
# Elastic Net is sensitive to the scale of features because of the L1 and L2 penalties
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_numeric)
X_test_scaled = scaler.transform(X_test_numeric)

# --- MODEL 4: Elastic Net Regression (L1 + L2) ---
# Combines L1 and L2 penalties to handle multicollinearity and perform feature selection
# l1_ratio=0.5 means an equal balance between Lasso and Ridge effects
elastic_reg = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)
elastic_reg.fit(X_train_scaled, y_train)
elastic_pred = elastic_reg.predict(X_test_scaled)

# Numerical Evaluation using the custom function
evaluate_regression("Elastic Net", y_test, elastic_pred, X_test_scaled)

In [ ]:
# Deleting model and prediction objects to free up RAM for ensemble models
del elastic_reg, elastic_pred, X_train_scaled, X_test_scaled
gc.collect()

### Support Vector Regressor (SVR)

### StandardScaler:
Mandatory for SVR to ensure all features contribute equally to distance calculations; otherwise, features with larger scales will bias the model, and Especially after the establishment Aggregation.

In [ ]:
# Scaling: SVR is extremely sensitive to the scale of data.
# We must scale features to have a mean of 0 and variance of 1.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_numeric)
X_test_scaled = scaler.transform(X_test_numeric)

### Methodology:

LinearSVR vs SVR (linear): LinearSVR is specifically optimized for large datasets, offering much faster computation and better memory efficiency than a standard SVR with a linear kernel.

RBF Kernel: Used to capture complex, non-linear patterns by mapping data into higher-dimensional space, allowing the model to find relationships that linear models cannot detect.

In [ ]:
# --- MODEL 5: Linear SVR ---
# LinearSVR is a faster implementation of SVR for linear kernels.
# Using dual=False when n_samples > n_features is recommended for better performance.
l_svr = LinearSVR(dual=False, loss='squared_epsilon_insensitive', max_iter=2000, random_state=42, tol=1e-5)
l_svr.fit(X_train_scaled, y_train)
l_svr_pred = l_svr.predict(X_test_scaled)

# Numerical Evaluation
evaluate_regression("Linear SVR", y_test, l_svr_pred, X_test_scaled)

# Deleting both the model and the large scaled arrays
del l_svr, l_svr_pred, X_train_scaled, X_test_scaled
gc.collect()

In [ ]:
# --- MODEL 6: SVR with RBF Kernel (Non-Linear) ---
# NOTE: RBF SVR has O(n_samples^2 * n_features) complexity.

# 1. Creating the safe sample (if not already created)
svr_limit = 50000
X_train_svr = X_train_scaled[:svr_limit]
y_train_svr = y_train[:svr_limit]
X_test_svr = X_test_scaled[:10000] # Smaller test set for RBF
y_test_svr = y_test[:10000]

# Using a 10,000 sample (X_train_svr) is mandatory to prevent RAM explosion.
rbf_svr = SVR(kernel='rbf', C=1.0, epsilon=0.1)

# 2.Training on the SAMPLE (X_train_svr)
rbf_svr.fit(X_train_svr, y_train_svr)

# 3.Predicting on the SAMPLE (X_test_svr)
rbf_svr_pred = rbf_svr.predict(X_test_svr)

# 4.Numerical Evaluation with the sample
evaluate_regression("RBF SVR (Sampled)", y_test_svr, rbf_svr_pred, X_test_svr)

# --- RAM Optimization ---
del rbf_svr, rbf_svr_pred, X_train_svr, y_train_svr, X_test_svr, y_test_svr
gc.collect()

### K-Nearest Neighbors (KNN)
Distance-Based Logic: I applied StandardScaler before training because KNN is highly sensitive to feature scales when calculating distances.

Metric Selection: I utilized Euclidean distance to find the most similar customer-product patterns.

Computational Efficiency: Used n_jobs=-1 for parallel processing and performed

In [ ]:
# --- MODEL: K-Nearest Neighbors (KNN) Regressor ---
# English Comment: KNN is memory-intensive because it stores all training instances.
# A 20k sample is used to allow multiple hyperparameter iterations without crashing.

# 1. Creating the specific sample for KNN
knn_limit = 20000
X_train_knn = X_train_scaled[:knn_limit]
y_train_knn = y_train[:knn_limit]
X_test_knn = X_test_scaled[:5000] # Smaller test set for faster distance calculation
y_test_knn = y_test[:5000]

# Hyperparameter tuning for KNN
k_values = [3, 7, 11]
metrics = ['euclidean', 'manhattan']

for k in k_values:
    for m in metrics:
        # Utilizing n_jobs=-1 for multi-core processing
        knn = KNeighborsRegressor(n_neighbors=k, metric=m, n_jobs=-1)

        # 2. Training on the SAMPLE (X_train_knn)
        # English Comment: Fitting on 20k samples instead of 10M
        knn.fit(X_train_knn, y_train_knn)

        # 3. Predicting on the SAMPLE (X_test_knn)
        knn_pred = knn.predict(X_test_knn)

        # 4. Performance Evaluation
        # English Comment: Passing the sample variables to ensure dimension consistency
        evaluate_regression(f"KNN (k={k}, metric={m})", y_test_knn, knn_pred, X_test_knn)

# --- CRITICAL RAM Optimization ---
del knn, knn_pred, X_train_knn, y_train_knn, X_test_knn, y_test_knn
gc.collect()

### Decision Tree Regressor


In [ ]:
# --- MODEL: Decision Tree Regressor ---
# Decision Trees are non-parametric and do not require feature scaling.
# We will test different depths to find the balance between underfitting and overfitting.

depths = [3, 5, 10, None] # None means the tree grows until all leaves are pure

for d in depths:

    dt_reg = DecisionTreeRegressor(max_depth=d, random_state=42)

    # Trees can handle larger datasets better than SVR/KNN,
    # but we use X_train_scaled to stay consistent with previous models
    dt_reg.fit(X_train_scaled, y_train)
    dt_pred = dt_reg.predict(X_test_scaled)

    # Numerical Evaluation
    depth_label = d if d is not None else "Unlimited"
    evaluate_regression(f"Decision Tree (max_depth={depth_label})", y_test, dt_pred, X_test_scaled)

    # Memory Management: Critical for tree-based models
    del dt_reg, dt_pred
    gc.collect()

### Random Forest Regressor


In [ ]:
# --- MODEL: Random Forest Regressor ---
# Note: Training on 10M rows with RF causes RAM crashes.
# We use a strategic sample (200k) to maintain performance and speed.

# 1. Creating a specific sample for Random Forest
rf_limit = 200000
X_train_rf = X_train_scaled[:rf_limit]
y_train_rf = y_train[:rf_limit]
X_test_rf = X_test_scaled[:20000] # Representative test sample
y_test_rf = y_test[:20000]

# Defining the model
# English Comment: n_jobs=-1 uses all CPU cores for faster training
rf_reg = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)

# 2. Training on the SAMPLE (X_train_rf)
# English Comment: Fitting on 200k samples for optimal training time
rf_reg.fit(X_train_rf, y_train_rf)

# 3. Predicting on the SAMPLE (X_test_rf)
rf_pred = rf_reg.predict(X_test_rf)

# 4. Numerical Evaluation
evaluate_regression("Random Forest Regressor (Sampled)", y_test_rf, rf_pred, X_test_rf)

# --- CRITICAL RAM Optimization ---
# English Comment: Deleting specific RF arrays to free memory
del rf_reg, rf_pred, X_train_rf, y_train_rf, X_test_rf, y_test_rf
gc.collect()

### LightGBM Regressor
Efficiency: "LightGBM is chosen over XGBoost due to its superior speed and lower memory consumption, which is critical for the high-dimensional Instacart dataset."

Hyperparameters: "We use a balanced learning_rate and num_leaves to ensure the model captures complex patterns without overfitting to noise."

Scalability: "The use of n_jobs=-1 ensures the boosting process is parallelized across all CPU cores."

In [ ]:
# --- MODEL: LightGBM Regressor ---
# Note: LightGBM is highly efficient for large datasets (10M+ rows).

# Convert X_test_scaled to DataFrame to match training feature names
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_train_numeric.columns)

# Defining the model
lgbm_reg = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42, n_jobs=-1)

# Training on the full scaled data
lgbm_reg.fit(X_train_scaled, y_train)

# Predicting on the scaled data
lgbm_pred = lgbm_reg.predict(X_test_scaled_df)

# Numerical Evaluation
evaluate_regression("LightGBM Regressor", y_test, lgbm_pred, X_test_scaled_df)

# CRITICAL RAM Optimization
del lgbm_reg, lgbm_pred, X_test_scaled_df
gc.collect()

### Evaluation Strategy:
Used MAE as the primary metric to provide a clear, business-ready interpretation of error in terms of "days."

In [ ]:
# --- FINAL PERFORMANCE COMPARISON ---

# Linear Variants
#evaluate_regression("Linear Regression (OLS)", y_test, ols_pred)
#evaluate_regression("Lasso (L1)", y_test, lasso_pred)
#evaluate_regression("Ridge (L2)", y_test, ridge_pred)
#evaluate_regression("Elastic Net", y_test, elastic_pred)

# Support Vector Machines (SVR)
#evaluate_regression("Linear SVR", y_test, l_svr_pred)
#evaluate_regression("RBF SVR", y_test, rbf_svr_pred)

# K-Nearest Neighbors
#evaluate_regression("KNN Regressor", y_test, knn_pred)

# Tree-based Models
#evaluate_regression("Decision Tree", y_test, dt_pred)
#evaluate_regression("Random Forest", y_test, rf_pred)
#evaluate_regression("LightGBM (Boosting)", y_test, lgb_pred)